# Experiment context

In [1]:
from recombinhunt.core.method import *
from recombinhunt.core.environment import Environment

env = Environment("environments/env_nextstrain_2023_03_30")
custom_environments = {
    "XBB": env.copy_with_exclusions(ignore_lineage=[l for l in env.included_lineages() if l.startswith("XBB") or any([sl in l for sl in ("EG", "EK", "EL", "EM", "EU", "FD", "FE", "FG", "FH", "FL")])]),
    "XBF": env.copy_with_exclusions(ignore_lineage=[l for l in env.included_lineages() if l.startswith("XBF")]),
    "XAY": env.copy_with_exclusions(ignore_lineage=[l for l in env.included_lineages() if l.startswith("XAY")]),
    "XP": env.copy_with_exclusions(ignore_lineage=[l for l in env.included_lineages() if l.startswith("XBB")])
}

# Preparation of test cases

- generation of consensus sequence for each recombinant case
- collection of PANGO-suggested breakpoint positions and recombination candidates to compare with RecombinHunt's output

In [2]:
import json
from recombinhunt.core.environment import PangoLineageHierarchy
from recombinhunt.validation.utils import *

# GROUND TRUTH
lh = PangoLineageHierarchy("demo/validation_data/alias_key.json")
cl = AssessedContributingLin("demo/validation_data/alias_key.json")


# NUMBER OF SEQUENCES PER CASE
with open("demo/demo_input_nextstrain/samples_total.json") as inp:
    number_of_sequences_by_rec_case = json.load(inp)


# CASE CLASSIFICATION
cases_1BP = [l for l in number_of_sequences_by_rec_case.keys() if BreakpointsLocation.breakpoints_num(l) == 1 and BreakpointsLocation.ith_breakpoint(l, 0)[0] != 0]
cases_2BP = [l for l in number_of_sequences_by_rec_case.keys() if BreakpointsLocation.breakpoints_num(l) == 2 and BreakpointsLocation.ith_breakpoint(l, 0)[0] != 0]
# collect cases with 3+ breakpoints and inaccurate candidates/breakpoints
cases_oth = [l for l in number_of_sequences_by_rec_case.keys() if BreakpointsLocation.breakpoints_num(l) == 0 or BreakpointsLocation.breakpoints_num(l) > 2 or BreakpointsLocation.ith_breakpoint(l, 0)[0] == 0]

# TARGET OF CASE (consensus sequence)
case2target = dict()
for case in cases_1BP + cases_2BP + cases_oth:
    this_case_sequences_str = []
    with open(f"demo/demo_input_nextstrain/samples_{case}.csv") as inp:
        inp.readline()
        for line in inp.readlines():
            name, true_lin, nuc_changes = line.rstrip('\n').split('\t')
            this_case_sequences_str.append(nuc_changes)
    case2target[case] = ["avg75", case, compute_75_perc_characterization(strings=this_case_sequences_str)]


def group_of_rec_case(rec_lineage, target_seq: list):
    if BreakpointsLocation.breakpoints_num(rec_lineage) == 2:
        return "2BP"
    elif BreakpointsLocation.breakpoints_num(rec_lineage) == 1:
        t_br_start, t_br_end = BreakpointsLocation.to_target_pos(BreakpointsLocation.ith_breakpoint(rec_lineage, 0), target_seq)
        t_len = len(target_seq)
        if t_br_start >= 10 and t_br_end <= (t_len - 10):
            return "1BP mid"
        elif t_br_start < 10:
            return "1BP 5'"
        else:
            return "1BP 3'"
    else:
        return "?"

cases_1BP_mid = [c for c in cases_1BP if group_of_rec_case(c, case2target[c][-1]) == '1BP mid']
cases_1BP_5p = [c for c in cases_1BP if group_of_rec_case(c, case2target[c][-1]) == '1BP 5\'']
cases_1BP_3p = [c for c in cases_1BP if group_of_rec_case(c, case2target[c][-1]) == '1BP 3\'']


print(len(cases_1BP), len(cases_2BP), len(cases_oth))


46 7 8


# Definition of output files

In [3]:
tabella_riassuntiva = []
print_case_detail = True
summary_table_file_path = "demo/demo_output_nextstrain/summary.md"
detailed_output_file_path = "demo/demo_output_nextstrain/detail.html"

In [4]:
import os

if os.path.exists(detailed_output_file_path):
    os.remove(detailed_output_file_path)

# Run of experiments and validation

In [5]:
from tqdm import tqdm
from recombinhunt.validation.case_analysis import CaseAnalysis

# CHOOSE ANALYSIS GROUP
#groups_of_cases, group_names, number_of_cases = (cases_2BP,), ("2BP",), len(cases_2BP)
#groups_of_cases, group_names, number_of_cases = (cases_1BP_mid, cases_1BP_5p, cases_1BP_3p), ("1BP_mid", "1BP_5p", "1BP_3p"), len(cases_1BP)
# groups_of_cases, group_names, number_of_cases = (cases_1BP_mid, cases_1BP_5p, cases_1BP_3p, cases_2BP), ("1BP_mid", "1BP_5p", "1BP_3p", "2BP"),  len(cases_1BP) + len(cases_2BP)
# groups_of_cases, group_names, number_of_cases = (cases_oth,), ("?",),  len(cases_oth)
groups_of_cases, group_names, number_of_cases = (cases_1BP_mid, cases_1BP_5p, cases_1BP_3p, cases_2BP, cases_oth), ("1BP mid", "1BP 5'", "1BP 3'", "2BP", "undefined"),  len(cases_1BP) + len(cases_2BP) + len(cases_oth)

# ANALYSIS  
issues = {
    "issue_0BP": [],    # cases labeled as recombinant by PANGO and non-recombinant by RecombinHunt
    "issue_2BP": [],    # cases labeled as 1BP recombinant by PANGO and 2BP by RecombinHunt
    "issue_1BP": [],    # 
    "issue_KO": [],
    "non_issue_OK": []
}

case_n = 0
met = False
with open(detailed_output_file_path, "a") as detailed_cases_outfile:
    with tqdm(total=number_of_cases) as progress:
        for group_of_cases, group_name in zip(groups_of_cases, group_names):
            for case in group_of_cases:
                try:
                    case_n += 1
                    progress.update()

                    # if not met and case != "XAV":
                    #     continue
                    # else:
                    #     met = True
                    #     pass
                    # if not case in ("XP"):
                    #     continue

                    # GROUND TRUTH DATA
                    N_seq = number_of_sequences_by_rec_case[case]
                    parent_lineage_list = cl.contributing_to(case)
                    breakpoints = BreakpointsLocation.all_breakpoints(case)
                    n_breakpoints = BreakpointsLocation.breakpoints_num(case)
                    nuc_changes = case2target[case][-1]

                    # EXPERIMENT
                    exp = Experiment(custom_environments.get(case, env))
                    exp.set_target(nuc_changes)
                    exp.run()

                    # COMPARE EXPERIMENT RESULT WITH GROUND TRUTH DATA
                    N_seq = f"(75%) {N_seq}"
                    ca = CaseAnalysis(exp, case, N_seq, case_n, group_name,
                                      parent_lineage_list, breakpoints,
                                      lh, cl)

                    ca.print_case_details(detailed_cases_outfile)

                    issues[ca.get_issue()].append(case)

                    tabella_riassuntiva.append(ca.analysis_table_row())

                except Exception as e:
                        print("ERROR", case, end=" ")
                        try:
                            print("Last analysed case", tabella_riassuntiva[-1][1], "case n° (1-based)", tabella_riassuntiva[-1][0])
                        except IndexError:
                            pass
                        raise e


100%|█████████████████████████████████████████████████████████████████████████████████████████| 61/61 [01:10<00:00,  1.16s/it]


# Output summary table

inspect summary table

In [6]:
display(pd.DataFrame(
    tabella_riassuntiva,
    columns=[
        'N°', 'Case', 'N_seq', 'N_mut', 'Class', 'GT*',
        'OK/KO', 'Initial_R_span', 'Gap (extr escl.)',
        'BC', 'Dir_L1', 'Rank',
        'Br_BC', 'Br_GT',
        'P_L1', 'P_L2',
        'ALT_C'
    ]
    )
    .set_index(('N°'))
)
print("0BP issue", len(issues["issue_0BP"]), issues["issue_0BP"])
print("2BP issue", len(issues["issue_2BP"]), issues["issue_2BP"])
print("1BP issue", len(issues["issue_1BP"]), issues["issue_1BP"])
print("KO", len(issues["issue_KO"]), issues["issue_KO"])
print("OK", len(issues["non_issue_OK"]), issues["non_issue_OK"])

,Case,N_seq,N_mut,Class,GT*,OK/KO,Initial_R_span,Gap (extr escl.),BC,Dir_L1,Rank,Br_BC,Br_GT,P_L1,P_L2,ALT_C
N°,,,,,,,,,,,,,,,,
1,XA,(75%) 33,36,1BP mid,B.1.177 + B.1.1.7,OK,"1-13,14-36",,B.1.177.18 + B.1.1.7,<<,2 1,13-14,12-14,1.24e-202,6.86e-101,"[], []"
2,XAD,(75%) 11,69,1BP mid,BA.2* + BA.1*,OK,"1-56,57-69",,BA.2 + BA.1,>>,1 1,56-57,55-57,4.11e-21,4.03e-170,"[BA.2.34], [BA.1.14.1, BA.1.14, BA.1.1.2, BA.1..."
3,XAE,(75%) 18,72,1BP mid,BA.2* + BA.1*,OK,"1-57,58-72",,BA.2 + BA.1,>>,1 1,57-58,55-58,1.11e-28,1.85e-168,"[], [BA.1.1, BA.1.14.1, BA.1.14]"
4,XAL,(75%) 13,67,1BP mid,BA.1* + BA.2*,OK,"1-17,18-67",,BA.1.1 + BA.2,<<,11 1,17-18,17-19,2.40e-104,1.96e-86,"[], []"
5,XAN,(75%) 36,72,1BP mid,BA.2* + BA.5.1,OK,"1-11,12-72",,BA.2 + BA.5.1.23,<<,1 2,11-12,21-28,2.19e-69,4.03e-17,"[BA.2.9, BA.2.12.1], [BA.5.1]"
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
57,XBN,(75%) 19,98,undefined,BA.2.75 + XBB.3,OK,"1-11,12-98",,BA.2.75.5 + XBB.3,<<,2 1,11-12,-,1.95e-197,6.67e-53,"[BA.2.75], [XBB.3.1]"
58,XBQ,(75%) 20,92,undefined,BA.5.2 + CJ.1,OK,"1-3,4-92",,BA.5.2.10 + CJ.1.1,<<,11 11,3-4,-,1.26e-321,1.15e-17,"[], [BM.1.1.1, BM.1.1]"
59,XBS,(75%) 15,88,undefined,BA.2.75 + BQ.1,KO,"1-3,6-29,30-88",3-30 -> 3-6,BQ.1.1 + BN.1 + BQ.1.1,<<,11 11 -,"5-6, 29-30",-,3.05e-68,2.17e-109,"[BN.1.3], []"


0BP issue 7 ['XAS', 'XAT', 'XAV', 'XB', 'XAR', 'XN', 'XAJ']
2BP issue 2 ['XBH', 'XBS']
1BP issue 3 ['XAK', 'XAZ', 'XAY']
KO 3 ['XBB', 'XM', 'XP']
OK 46 ['XA', 'XAD', 'XAE', 'XAL', 'XAN', 'XAP', 'XBD', 'XBE', 'XBF', 'XBG', 'XBJ', 'XBM', 'XBP', 'XBR', 'XBW', 'XJ', 'XV', 'XY', 'XZ', 'XAA', 'XAB', 'XAF', 'XAG', 'XAM', 'XAU', 'XE', 'XF', 'XG', 'XH', 'XL', 'XQ', 'XR', 'XS', 'XU', 'XW', 'XAH', 'XAC', 'XBL', 'XBT', 'XBU', 'XD', 'XBK', 'XBN', 'XBQ', 'XBV', 'XCA']


write summary to file

In [7]:
import pandas as pd

(pd.DataFrame(
    tabella_riassuntiva,
    columns=[
        'N°', 'Case', 'N_seq', 'N_mut', 'Class', 'GT*',
        'OK/KO', 'Initial_R_span', 'Gap history (estr. escl.)',
        'BC', 'Dir_L1', 'Rank',
        'Br_BC', 'Br_GT',
        'P_L1', 'P_L2',
        'ALT_C'
    ]
    )
    .set_index(('N°'))
    .to_markdown(summary_table_file_path)
)
with open(summary_table_file_path, "a") as out:
    out.write('\n\n')
    print("0BP issue", len(issues["issue_0BP"]), issues["issue_0BP"], file=out)
    print("2BP issue", len(issues["issue_2BP"]), issues["issue_2BP"], file=out)
    print("1BP issue", len(issues["issue_1BP"]), issues["issue_1BP"], file=out)
    print("KO", len(issues["issue_KO"]), issues["issue_KO"], file=out)
    print("OK", len(issues["non_issue_OK"]), issues["non_issue_OK"], file=out)